<a href="https://colab.research.google.com/github/pihchikk/machine-learning-to-predict-the-oxidizaility-of-organic-carbon-in-surface-soil/blob/main/data%20preprocessing/notebooks/Tambov_phenology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Auth

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-shrgnn')

### ROI

In [ ]:
from pyproj import Proj, transform

# Define the EPSG code for EPSG:32637 (UTM Zone 37N) and WGS84
epsg_32637 = Proj(init='epsg:32637')
wgs84 = Proj(init='epsg:4326')

# Coordinates in EPSG:32637
x1, y1 = 647500, 5766000
x2, y2 = 651000, 5770000

# Convert EPSG:32637 coordinates to WGS84
lon1, lat1 = transform(epsg_32637, wgs84, x1, y1)
lon2, lat2 = transform(epsg_32637, wgs84, x2, y2)

region = ee.Geometry.Polygon(
  [[[lon1, lat2],
    [lon1, lat1],
    [lon2, lat1],
    [lon2, lat2]]], None, False)


### Date ranges (half-month averaging)

In [ ]:
from datetime import datetime

# Define the list of months
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'][2:12]

# Function to get the maximum day of the month
def max_day(month):
    if month in ['01', '03', '05', '07', '08', '10', '12']:
        return '31'
    elif month == '02':
        return '28'  # Assuming non-leap year for simplicity
    else:
        return '30'

# Create the list comprehension for first and second halves separately
first_half = [{'start_date': f'2021-{month}-01', 'end_date': f'2021-{month}-16'} for month in months]
second_half = [{'start_date': f'2021-{month}-16', 'end_date': f'2021-{month}-{max_day(month)}'} for month in months]

# Combine the first and second halves
date_ranges = first_half + second_half

# Convert date strings to datetime objects and sort
date_ranges.sort(key=lambda x: datetime.strptime(x['start_date'], '%Y-%m-%d'))

# Print the sorted date ranges
print(date_ranges)

### Decadal ranges (decadal averaging)

In [ ]:
from datetime import datetime

def get_date_ranges(year):
    # Define the list of months
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

    # Function to get the maximum day of the month
    def max_day(month):
        if month in ['01', '03', '05', '07', '08', '10', '12']:
            return '31'
        elif month == '02':
            # Assuming non-leap year for simplicity
            return '28' if not year % 4 == 0 or (year % 100 == 0 and year % 400 != 0) else '29'
        else:
            return '30'

    # Create the list comprehension for first, second, and third decades separately
    first_decade = [{'start_date': f'{year}-{month}-01', 'end_date': f'{year}-{month}-10'} for month in months]
    second_decade = [{'start_date': f'{year}-{month}-11', 'end_date': f'{year}-{month}-20'} for month in months]
    third_decade = [{'start_date': f'{year}-{month}-21', 'end_date': f'{year}-{month}-{max_day(month)}'} for month in months]

    # Combine the first, second, and third decades
    date_ranges = first_decade + second_decade + third_decade

    # Convert date strings to datetime objects and sort
    date_ranges.sort(key=lambda x: datetime.strptime(x['start_date'], '%Y-%m-%d'))

    return date_ranges

# Example usage:
year = 2021
date_ranges = get_date_ranges(year)
print(date_ranges)


### Phenology Indicies

In [ ]:
import ee
import geemap
import folium

# Initialize the Earth Engine library
ee.Initialize()

def computeEVI2(image):
    """Compute the EVI2 index for a given image."""
    NIR = image.select('B8')  # Assuming NIR band is labeled as B8
    red = image.select('B4')  # Assuming red band is labeled as B4
    DVI = NIR.subtract(red).rename('DVI')
    EVI2 = ee.Image(2.5).multiply(DVI) \
        .divide(NIR.add(ee.Image(2.4).multiply(red)).add(ee.Image(1))).rename('EVI2')
    return image.addBands(EVI2)

def compute_yearly_sum_EVI2(year):
    """Compute the pixel-wise sum of EVI2 values >= 0.2 for a given year."""
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'

    # Load Sentinel-2 image collection for the given year
    s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate(start_date, end_date).filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 80))
    s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(region).filterDate(start_date, end_date)

    # Join S2 SR with cloud probability dataset to add cloud mask
    s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
        primary=s2Sr,
        secondary=s2Clouds,
        condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
    )

    def maskClouds(img):
        clouds = ee.Image(img.get('cloud_mask')).select('probability')
        isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
        return img.updateMask(isNotCloud)

    def maskEdges(s2_img):
        return s2_img.updateMask(
            s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

    s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)

    # Compute EVI2 for each image and mask EVI2 values below 0.2
    def mask_evi2_below_threshold(image):
        evi2_image = computeEVI2(image)
        evi2 = evi2_image.select('EVI2')
        mask = evi2.gte(0.2)
        return evi2_image.updateMask(mask)

    evi2_collection = s2CloudMasked.map(mask_evi2_below_threshold)

    # Sum the EVI2 values for each pixel
    evi2_sum = evi2_collection.select('EVI2').sum().rename('EVI2').clip(region)

    return evi2_sum

# Compute the yearly EVI2 sums for the years 2019-2023 and save them in a list
years = [2019, 2020, 2021, 2022, 2023]
TP_yearly_list = []

for year in years:
    evi2_sum = compute_yearly_sum_EVI2(year)
    TP_yearly_list.append([evi2_sum, year])
    print(f"Computed EVI2 sum for year {year}")

### VIS Phenology

In [ ]:
import folium
import ee

k = 1
j = 2019+k

# Define the visualization parameters
SOSVIS = {'min': 0, 'max': 365, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red']}
EOSVIS = {'min': 0, 'max': 365, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red']}
TPVIS = {'min': 0, 'max': 2500, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red']}

IndVIS = {'min': 0, 'max': 2.5, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red']}
curveVIS = {'min': 0, 'max': 0.05, 'bands':'EVI2', 'palette': ['cyan', 'green', 'yellow', 'orange', 'red']}

# Create a Folium map centered around your region of interest
map = folium.Map(location=[52.03, 41.16], zoom_start=10)

# Get the map ID for the EVI2 image
'''
map_id = result_image.getMapId(SOSVIS)

# Add the EVI2 image as a tile layer to the map
folium.TileLayer(
    tiles=map_id['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='EVI2 Masked',
).add_to(map)
'''
map_id2 = SOS_yearly_list[k][1].getMapId(SOSVIS)

# Add the EVI2 image as a tile layer to the map
folium.TileLayer(
    tiles=map_id2['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'SOS {j} median',
).add_to(map)

map_id3 = EOS_yearly_list[k][1].getMapId(SOSVIS)

# Add the EVI2 image as a tile layer to the map
folium.TileLayer(
    tiles=map_id3['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'EOS {j} Masked median',
).add_to(map)

map_id5 = Seasamp_yearly_list[k][1].getMapId(IndVIS)

# Add the EVI2 image as a tile layer to the map
folium.TileLayer(
    tiles=map_id5['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'seasonal amplitude {j}',
).add_to(map)

map_id6 = greenup_yearly_list[k][1].getMapId(curveVIS)

# Add the EVI2 image as a tile layer to the map
folium.TileLayer(
    tiles=map_id6['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'greenup slope {j}',
).add_to(map)

map_id7 = greendown_yearly_list[k][1].getMapId(curveVIS)

# Add the EVI2 image as a tile layer to the map
folium.TileLayer(
    tiles=map_id7['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'greendown slope {j}',
).add_to(map)

map_id8 = TP_yearly_list[k][0].getMapId(TPVIS)

# Add the EVI2 image as a tile layer to the map
folium.TileLayer(
    tiles=map_id8['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'Total Seasonal Productivity {j}',
).add_to(map)

map_id9 = season_length_list[k][0].getMapId(EOSVIS)

# Add the EVI2 image as a tile layer to the map
folium.TileLayer(
    tiles=map_id9['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'Season Duration {j}',
).add_to(map)



roi_geojson = region.getInfo()
folium.GeoJson(roi_geojson, name='ROI').add_to(map)

# Add layer control to the map
folium.LayerControl().add_to(map)

# Display the map
map


### Compute and export PPI for given START_DATE, END_DATE, least cloudy

In [ ]:
import ee

# Define the desired date range
START_DATE = ee.Date('2021-10-01')
END_DATE = ee.Date('2021-12-01')

# Filter the image collection by date range
filtered_collection = s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate(START_DATE, END_DATE)

# Sort the filtered collection by cloud cover
sorted_collection = filtered_collection.sort('CLOUDY_PIXEL_PERCENTAGE')

# Select the image with the lowest cloud cover
selected_image = ee.Image(sorted_collection.first())
date = selected_image.date().format('YYYY-MM-dd').getInfo()

selected_image = compute_PPI(selected_image)
# Define the export parameters

export_params = {
    'image': selected_image.select(['PPI']),
    'description': f'lowest_cloud_cover_image {date}',
    'folder': 'GEE_images',  # Replace 'your_folder_name' with your desired folder name
    'scale': 10,  # Adjust the scale as needed
    'region': region,  # Define the region of interest
}

# Export the image to your Google Drive
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Print the task status
print('Exporting image with lowest cloud cover to Google Drive...')


## SOS Yearly (2021) Median + Median images decadal calculation

In [ ]:
import geemap
import folium
import numpy as np

s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate('2018-01-01', '2023-12-31')
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
MAX_CLOUD_PROBABILITY = 65

#EVI threshold
threshold = 0.35

def map_SOS_EOS(image):
    SOS, EOS = find_SOS_EOS(image, threshold=0.2)
    return image.addBands(SOS.rename('SOS')).addBands(EOS.rename('EOS'))

#DVImax_yearly = compute_yearly_DVImax(s2Sr)

# Create a map centered at the region of interest
map = folium.Map(location=[52.03, 41.16], zoom_start=10)

# Function to compute NDSI
def computeNDSI(img):
    ndsi = img.normalizedDifference(['B3', 'B11']).rename('NDSI')
    ndsi2 = img.normalizedDifference(['B3', 'B12']).rename('NDSI2')
    return img.addBands(ndsi).addBands(ndsi2)

# Function to mask clouds
def maskClouds(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)

# Function to mask edges
def maskEdges(s2_img):
    return s2_img.updateMask(
        s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

SOS_median = ee.Image.constant(0)

full_EVI2_collection = ee.ImageCollection([])

for date_range in date_ranges: #phenology season
    START_DATE = ee.Date(date_range['start_date'])
    END_DATE = ee.Date(date_range['end_date'])
    middle_date = START_DATE.advance((END_DATE.difference(START_DATE, 'days')).divide(2), 'days')

    # Filter images by date range and region
    criteria_a = ee.Filter.And(
        ee.Filter.bounds(region),
        ee.Filter.date(START_DATE, END_DATE))
    criteria_b = ee.Filter.And(
        ee.Filter.bounds(region),
        ee.Filter.date(START_DATE, END_DATE),
        ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 40))

    s2Sr_filtered = s2Sr.filter(criteria_b)
    s2Clouds_filtered = s2Clouds.filter(criteria_a)

    # Join S2 SR with cloud probability dataset to add cloud mask
    s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
        primary=s2Sr_filtered,
        secondary=s2Clouds_filtered,
        condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
    )
    s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)
    s2CloudMaskedWithNDSI = s2CloudMasked.map(computeNDSI)
    PPIvis = {'min': -2, 'max': 2, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red'], 'bands': ['PPI']}
    rgbVis = {'min': 0, 'max': 2500, 'bands': ['B4', 'B3', 'B2']}

    images = s2CloudMasked.toList(s2CloudMaskedWithNDSI.size())

    EVI2_images = []

    try:
        for i in range(images.size().getInfo()):

            image = ee.Image(images.get(i))
            date = image.date().format('YYYY-MM-dd').getInfo()
            # Compute PPI for the image
            EVI2_image = computeEVI2(image)
            EVI2_images.append(EVI2_image)

        EVI2_collection = ee.ImageCollection(EVI2_images)


        # Define the threshold for the seasonal amplitude
        threshold = seas_amp_median.multiply(0.25)

        # Compute the median EVI2 image for the entire date range
        median_EVI2_image = EVI2_collection.median()

        # Mask the median EVI2 image based on the thresholded seasonal amplitude
        masked_median_EVI2_image = median_EVI2_image.updateMask(seas_amp_median.gt(threshold))

        masked_median_EVI2_image = masked_median_EVI2_image.set('system:time_start', middle_date.millis())
        day_of_year = ee.Date(masked_median_EVI2_image.getInfo()['properties']['system:time_start']).difference(ee.Date('2021-01-01'), 'days').round()

        # Create a constant image representing the day of year
        constant_day = ee.Image.constant(day_of_year)

        # Update the result image to store the start-of-season day for each pixel
        SOS_median = SOS_median.where(SOS_median.eq(0).And(masked_median_EVI2_image.select('EVI2').gt(threshold)), constant_day)


        full_EVI2_collection = full_EVI2_collection.merge(median_EVI2_image)

    except:
      print('exception')
      pass






## EOS 2021 non-median

In [ ]:
import geemap
import folium
import numpy as np

s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate('2018-01-01', '2023-12-31')
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
MAX_CLOUD_PROBABILITY = 65

#EVI threshold
threshold = 0.35

def map_SOS_EOS(image):
    SOS, EOS = find_SOS_EOS(image, threshold=0.2)
    return image.addBands(SOS.rename('SOS')).addBands(EOS.rename('EOS'))

#DVImax_yearly = compute_yearly_DVImax(s2Sr)

# Create a map centered at the region of interest
map = folium.Map(location=[52.03, 41.16], zoom_start=10)

# Function to compute NDSI
def computeNDSI(img):
    ndsi = img.normalizedDifference(['B3', 'B11']).rename('NDSI')
    ndsi2 = img.normalizedDifference(['B3', 'B12']).rename('NDSI2')
    return img.addBands(ndsi).addBands(ndsi2)

# Function to mask clouds
def maskClouds(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)

# Function to mask edges
def maskEdges(s2_img):
    return s2_img.updateMask(
        s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

SOS_median = ee.Image.constant(0)

full_EVI2_collection = ee.ImageCollection([])

for date_range in date_ranges[9:30]: #phenology season
    START_DATE = ee.Date(date_range['start_date'])
    END_DATE = ee.Date(date_range['end_date'])
    middle_date = START_DATE.advance((END_DATE.difference(START_DATE, 'days')).divide(2), 'days')

    # Filter images by date range and region
    criteria_a = ee.Filter.And(
        ee.Filter.bounds(region),
        ee.Filter.date(START_DATE, END_DATE))
    criteria_b = ee.Filter.And(
        ee.Filter.bounds(region),
        ee.Filter.date(START_DATE, END_DATE),
        ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 40))

    s2Sr_filtered = s2Sr.filter(criteria_b)
    s2Clouds_filtered = s2Clouds.filter(criteria_a)

    # Join S2 SR with cloud probability dataset to add cloud mask
    s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
        primary=s2Sr_filtered,
        secondary=s2Clouds_filtered,
        condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
    )
    s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)
    s2CloudMaskedWithNDSI = s2CloudMasked.map(computeNDSI)
    PPIvis = {'min': -2, 'max': 2, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red'], 'bands': ['PPI']}
    rgbVis = {'min': 0, 'max': 2500, 'bands': ['B4', 'B3', 'B2']}

    if s2CloudMaskedWithNDSI.size().getInfo() > 0:

        images = s2CloudMasked.toList(s2CloudMaskedWithNDSI.size())

        EVI2_images = []

        for i in range(images.size().getInfo()):

            image = ee.Image(images.get(i))
            date = image.date().format('YYYY-MM-dd').getInfo()
            # Compute PPI for the image
            EVI2_image = computeEVI2(image)



            # Define the threshold for the seasonal amplitude
            threshold = seas_amp_median.multiply(0.25)

            # Mask the median EVI2 image based on the thresholded seasonal amplitude
            masked_EVI2_image = EVI2_image.updateMask(seas_amp_median.gt(threshold))

            masked_EVI2_image = masked_EVI2_image.set('system:time_start', middle_date.millis())
            day_of_year = ee.Date(masked_EVI2_image.getInfo()['properties']['system:time_start']).difference(ee.Date('2021-01-01'), 'days').round()

            # Create a constant image representing the day of year
            constant_day = ee.Image.constant(day_of_year)

            # Update the result image to store the start-of-season day for each pixel
            SOS_median = SOS_median.where(SOS_median.eq(0).And(masked_EVI2_image.select('EVI2').gt(threshold)), constant_day)

            '''
            full_EVI2_collection = full_EVI2_collection.merge(median_EVI2_image)
            '''






## EOS yearly (2021) median

In [ ]:
import geemap
import folium
import numpy as np

s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate('2018-01-01', '2023-12-31')
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
MAX_CLOUD_PROBABILITY = 65

def map_SOS_EOS(image):
    SOS, EOS = find_SOS_EOS(image, threshold=0.2)
    return image.addBands(SOS.rename('SOS')).addBands(EOS.rename('EOS'))

#DVImax_yearly = compute_yearly_DVImax(s2Sr)

# Create a map centered at the region of interest
map = folium.Map(location=[52.03, 41.16], zoom_start=10)

# Function to compute NDSI
def computeNDSI(img):
    ndsi = img.normalizedDifference(['B3', 'B11']).rename('NDSI')
    ndsi2 = img.normalizedDifference(['B3', 'B12']).rename('NDSI2')
    return img.addBands(ndsi).addBands(ndsi2)

# Function to mask clouds
def maskClouds(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)

# Function to mask edges
def maskEdges(s2_img):
    return s2_img.updateMask(
        s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

EOS_median = ee.Image.constant(0)

for date_range in date_ranges[12:]: #phenology season
    START_DATE = ee.Date(date_range['start_date'])
    END_DATE = ee.Date(date_range['end_date'])
    middle_date = START_DATE.advance((END_DATE.difference(START_DATE, 'days')).divide(2), 'days')

    # Filter images by date range and region
    criteria_a = ee.Filter.And(
        ee.Filter.bounds(region),
        ee.Filter.date(START_DATE, END_DATE))
    criteria_b = ee.Filter.And(
        ee.Filter.bounds(region),
        ee.Filter.date(START_DATE, END_DATE),
        ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 40))

    s2Sr_filtered = s2Sr.filter(criteria_b)
    s2Clouds_filtered = s2Clouds.filter(criteria_a)

    # Join S2 SR with cloud probability dataset to add cloud mask
    s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
        primary=s2Sr_filtered,
        secondary=s2Clouds_filtered,
        condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
    )
    s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)
    s2CloudMaskedWithNDSI = s2CloudMasked.map(computeNDSI)
    PPIvis = {'min': -2, 'max': 2, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red'], 'bands': ['PPI']}
    rgbVis = {'min': 0, 'max': 2500, 'bands': ['B4', 'B3', 'B2']}

    images = s2CloudMasked.toList(s2CloudMaskedWithNDSI.size())

    EVI2_images = []

    try:
        for i in range(images.size().getInfo()):

            image = ee.Image(images.get(i))
            date = image.date().format('YYYY-MM-dd').getInfo()
            # Compute PPI for the image
            EVI2_image  = computeEVI2(image)
            EVI2_images.append(EVI2_image)

        EVI2_collection = ee.ImageCollection(EVI2_images)

        threshold = seas_amp_median.multiply(0.15)

        # Calculate the median EVI2 image for the entire date range
        median_EVI2_image = EVI2_collection.median()

        # Mask the median EVI2 image
        masked_median_EVI2_image = median_EVI2_image.updateMask(seas_amp_median.gt(threshold))

        masked_median_EVI2_image = masked_median_EVI2_image.set('system:time_start', middle_date.millis())
        day_of_year = ee.Date(masked_median_EVI2_image.getInfo()['properties']['system:time_start']).difference(ee.Date('2021-01-01'), 'days').round()
        constant_day = ee.Image.constant(day_of_year)

        # Update the result image
        EOS_median = EOS_median.where((masked_median_EVI2_image.select('EVI2').gt(threshold)), constant_day)

        negative_condition = EOS_median.lte(0)

        # Get the maximum EOS value
        max_EOS_value = EOS_median.reduceRegion(
            reducer=ee.Reducer.max(),
            geometry=region,
            scale=10
        ).get('constant')

        max_EOS_constant = ee.Image.constant(max_EOS_value)

        # Set EOS_median to the maximum EOS value where it is negative
        EM = EOS_median.where(negative_condition, max_EOS_constant)

    except:
        pass

## Seasonal amplitude yearly (2021)

In [ ]:
s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate('2021-01-01', '2021-12-31').filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 50))
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(region).filterDate('2021-01-01', '2021-12-31')
MAX_CLOUD_PROBABILITY = 65

s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
        primary=s2Sr,
        secondary=s2Clouds,
        condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
    )
s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)

def seasonal_amplitude(collection):
    EVI_collection = collection.map(computeEVI2)
    EVImax = EVI_collection.select('EVI2').reduce(ee.Reducer.max()).rename('EVImax')
    EVImin = EVI_collection.select('EVI2').reduce(ee.Reducer.min()).rename('EVImin')


    return EVImax.subtract(EVImin)

seas_amp = seasonal_amplitude(s2Sr)
seas_amp_median = seasonal_amplitude(full_EVI2_collection)

## Slope of the Green-up Period (2021)

In [ ]:
import folium
import ee

# Initialize the Earth Engine library
ee.Initialize()

# Define the region of interest
region = region

# Define the date ranges around 15th April and 15th May
april_start = ee.Date('2021-04-08')
april_end = ee.Date('2021-04-22')
may_start = ee.Date('2021-05-08')
may_end = ee.Date('2021-05-22')

# Define the image collection and filter by date and cloud cover
s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filter(ee.Filter.date(april_start, april_end)) \
    .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 50)) \
    .sort('CLOUDY_PIXEL_PERCENTAGE', True) \
    .limit(1)

s2SrMay = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filter(ee.Filter.date(may_start, may_end)) \
    .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 50)) \
    .sort('CLOUDY_PIXEL_PERCENTAGE', True) \
    .limit(1)

# Compute PPI for April
s2Sr_April = s2Sr.map(computeEVI2).median()

# Compute PPI for May
s2Sr_May = s2SrMay.map(computeEVI2).median()

# Compute the difference in PPI values between April and May
diff_PPI = s2Sr_May.subtract(s2Sr_April)

april_date = s2Sr.first().date()
may_date = s2SrMay.first().date()

# Compute the number of days between the images
num_days = may_date.difference(april_date, 'days')

# Divide the difference in PPI values by the number of days between the two dates
slope_greenup = diff_PPI.divide(num_days)

# Create a color map from green to red
cmap = ['00FF00', '#ffff00', 'FF0000']

# Create a map centered at the region of interest
map = folium.Map(location=[0, 0], zoom_start=2)

# Add the region of interest to the map
folium.GeoJson(
    data=region.getInfo(),
    name='Region of Interest',
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'blue', 'weight': 2},
    tooltip='Region of Interest'
).add_to(map)

# Add the slope_greenup image to the map
mapid = slope_greenup.select(['EVI2']).getMapId({'min': 0, 'max': 0.05, 'palette': cmap})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='slope_greenup',
).add_to(map)

# Display the map
map

## Slope of the Green-up Period 2019-2023

In [ ]:
import folium
import ee

# Initialize the Earth Engine library
ee.Initialize()

# Define the date ranges around 15th April and 15th May
april_start_base = ee.Date('2021-04-02')
april_end_base = ee.Date('2021-04-25')
may_start_base = ee.Date('2021-05-06')
may_end_base = ee.Date('2021-05-29')

# Create a color map from green to red
cmap = ['00FF00', '#ffff00', 'FF0000']

# Create a map centered at the region of interest
map = folium.Map(location=[52.03, 41.16], zoom_start=10)

# Initialize an empty image to accumulate slope_greenup values
slope_greenup_accumulated = ee.Image.constant(0)

# Initialize a counter for the number of years
num_years = 0

greenup_yearly_list = []

# Loop over years from 2018 to 2024
for year in range(2019, 2024):
    # Increment the start dates by one year
    april_start = april_start_base.advance(year - 2021, 'year')
    april_end = april_end_base.advance(year - 2021, 'year')
    may_start = may_start_base.advance(year - 2021, 'year')
    may_end = may_end_base.advance(year - 2021, 'year')

    may_end_str = may_end.format('YYYY-MM-dd').getInfo()

    # Define the image collection and filter by date and cloud cover for April
    s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(region) \
        .filter(ee.Filter.date(april_start, april_end)) \
        .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 50)) \
        .sort('CLOUDY_PIXEL_PERCENTAGE', True) \
        .limit(1)

    # Define the image collection and filter by date and cloud cover for May
    s2SrMay = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(region) \
        .filter(ee.Filter.date(may_start, may_end)) \
        .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 50)) \
        .sort('CLOUDY_PIXEL_PERCENTAGE', True) \
        .limit(1)

    # Compute PPI for April
    s2Sr_April = s2Sr.map(computeEVI2).median()

    # Compute PPI for May
    s2Sr_May = s2SrMay.map(computeEVI2).median()

    # Compute the difference in PPI values between April and May
    diff_PPI = s2Sr_May.subtract(s2Sr_April)

    # Get the dates of the images in s2Sr and s2SrMay
    april_date = s2Sr.first().date()
    may_date = s2SrMay.first().date()

    # Compute the number of days between the images
    num_days = may_date.difference(april_date, 'days')
    print(year)

    # Divide the difference in PPI values by the number of days between the two dates
    slope_greenup = diff_PPI.divide(num_days)
    # Add the slope_greenup image to the map with year and dates in the title
    mapid = slope_greenup.select(['EVI2']).getMapId({'min': 0, 'max': 0.05, 'palette': cmap})
    folium.TileLayer(
        tiles=mapid['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        overlay=True,
        name=f'slope_greenup_{year}_{april_date.format("YYYY-MM-dd").getInfo()}_{may_date.format("YYYY-MM-dd").getInfo()}',
    ).add_to(map)

    slope_greenup_accumulated = slope_greenup_accumulated.add(slope_greenup)

    num_years += 1

    greenup_yearly_list.append([year, slope_greenup])

mean_slope_greenup = slope_greenup_accumulated.divide(num_years)

mapid2 = mean_slope_greenup.select(['EVI2']).getMapId({'min': 0, 'max': 0.05, 'palette': cmap})
folium.TileLayer(
    tiles=mapid2['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name=f'mean_slope_greenup_all_years',
).add_to(map)


roi_geojson = region.getInfo()
folium.GeoJson(roi_geojson, name='ROI').add_to(map)

folium.LayerControl().add_to(map)

map

## Slope of the Green-down Period (2021)

In [ ]:
import folium
import ee

# Initialize the Earth Engine library
ee.Initialize()

# Define the region of interest
region = region

# Define the date ranges around 15th April and 15th May
november_start = ee.Date('2021-10-11')
november_end = ee.Date('2021-11-25')
december_start = ee.Date('2021-12-01')
december_end = ee.Date('2021-12-31')

# Define the image collection and filter by date and cloud cover
s2Sr_november_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filter(ee.Filter.date(april_start, april_end)) \
    .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 50)) \
    .sort('CLOUDY_PIXEL_PERCENTAGE', True) \
    .limit(1)

s2Sr_december_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filter(ee.Filter.date(may_start, may_end)) \
    .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 50)) \
    .sort('CLOUDY_PIXEL_PERCENTAGE', True) \
    .limit(1)

# Compute PPI for April
s2Sr_november = s2Sr.map(computeEVI2).median()

# Compute PPI for May
s2Sr_december = s2SrMay.map(computeEVI2).median()

# Compute the difference in PPI values between April and May
diff_PPI = s2Sr_november.subtract(s2Sr_december)

november_date = s2Sr_november_col.first().date()
december_date = s2Sr_december_col.first().date()

# Compute the number of days between the images
num_days = december_date.difference(november_date, 'days')

# Divide the difference in PPI values by the number of days between the two dates
slope_greendown = diff_PPI.divide(num_days)

# Create a color map from green to red
cmap = ['00FF00', '#ffff00', 'FF0000']

# Create a map centered at the region of interest
map = folium.Map(location=[0, 0], zoom_start=2)

# Add the region of interest to the map
folium.GeoJson(
    data=region.getInfo(),
    name='Region of Interest',
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'blue', 'weight': 2},
    tooltip='Region of Interest'
).add_to(map)

# Add the slope_greenup image to the map
mapid = slope_greendown.select(['EVI2']).getMapId({'min': 0, 'max': 0.1, 'palette': cmap})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='slope_greenup',
).add_to(map)

# Display the map
map

## Slope of the Green-down Period 2019-2023

In [ ]:
import folium
import ee

# Initialize the Earth Engine library
ee.Initialize()

# Define the region of interest
region = region

# Define the date ranges around 15th November and 15th December
november_start_base = ee.Date('2021-09-15')
november_end_base = ee.Date('2021-10-15')
december_start_base = ee.Date('2021-11-01')
december_end_base = ee.Date('2021-12-01')

# Create a color map from green to red
cmap = ['00FF00', '#ffff00', 'FF0000']

# Create a map centered at the region of interest
map = folium.Map(location=[52.03, 41.16], zoom_start=6)

# Initialize an empty image to accumulate slope_greendown values
slope_greendown_accumulated = ee.Image.constant(0)

# Initialize a counter for the number of years
num_years = 0

greendown_yearly_list = []

# Loop over years from 2019 to 2023
for year in range(2019, 2024):
    # Increment the start dates by one year
    november_start = november_start_base.advance(year - 2021, 'year')
    november_end = november_end_base.advance(year - 2021, 'year')
    december_start = december_start_base.advance(year - 2021, 'year')
    december_end = december_end_base.advance(year - 2021, 'year')

    # Define the image collection and filter by date and cloud cover for November
    s2Sr_november_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(region) \
        .filter(ee.Filter.date(november_start, november_end)) \
        .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 65)) \
        .sort('CLOUDY_PIXEL_PERCENTAGE', True) \
        .limit(1)

    # Define the image collection and filter by date and cloud cover for December
    s2Sr_december_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(region) \
        .filter(ee.Filter.date(december_start, december_end)) \
        .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 65)) \
        .sort('CLOUDY_PIXEL_PERCENTAGE', True) \
        .limit(1)

    # Compute PPI for November
    s2Sr_november = s2Sr_november_col.map(computeEVI2).median()

    # Compute PPI for December
    s2Sr_december = s2Sr_december_col.map(computeEVI2).median()

    # Compute the difference in PPI values between November and December
    diff_PPI = s2Sr_november.subtract(s2Sr_december)

    # Get the dates of the images in s2Sr_november and s2Sr_december
    november_date = s2Sr_november_col.first().date()
    december_date = s2Sr_december_col.first().date()

    # Compute the number of days between the images
    num_days = december_date.difference(november_date, 'days')

    # Divide the difference in PPI values by the number of days between the two dates
    slope_greendown = diff_PPI.divide(num_days)

    # Add the slope_greendown image to the map with year and dates in the title
    mapid = slope_greendown.select(['EVI2']).getMapId({'min': 0, 'max': 0.05, 'palette': cmap})
    folium.TileLayer(
        tiles=mapid['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        overlay=True,
        name=f'slope_greendown_{year}_{november_date.format("YYYY-MM-dd").getInfo()}_{december_date.format("YYYY-MM-dd").getInfo()}',
    ).add_to(map)

    slope_greendown_accumulated = slope_greendown_accumulated.add(slope_greendown)

    num_years += 1

    greendown_yearly_list.append([year, slope_greendown])

# Compute the mean slope_greendown over all years
mean_slope_greendown = slope_greendown_accumulated.divide(num_years)

# Add the mean slope_greendown image to the map
mapid2 = mean_slope_greendown.select(['EVI2']).getMapId({'min': 0, 'max': 0.05, 'palette': cmap})
folium.TileLayer(
    tiles=mapid2['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name=f'mean_slope_greendown_all_years',
).add_to(map)

# Add the region of interest to the map
folium.GeoJson(
    data=region.getInfo(),
    name='Region of Interest',
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'blue', 'weight': 2},
    tooltip='Region of Interest'
).add_to(map)

# Add layer control to the map
folium.LayerControl().add_to(map)

# Display the map
map

## SOS Medians 2019-2023

In [ ]:
import geemap
import folium
import numpy as np

#BLOCK1

s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate('2018-01-01', '2023-12-31')
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
MAX_CLOUD_PROBABILITY = 65

def map_SOS_EOS(image):
    SOS, EOS = find_SOS_EOS(image, threshold=0.2)
    return image.addBands(SOS.rename('SOS')).addBands(EOS.rename('EOS'))

#DVImax_yearly = compute_yearly_DVImax(s2Sr)

# Create a map centered at the region of interest
map = folium.Map(location=[52.03, 41.16], zoom_start=10)

# Function to compute NDSI
def computeNDSI(img):
    ndsi = img.normalizedDifference(['B3', 'B11']).rename('NDSI')
    ndsi2 = img.normalizedDifference(['B3', 'B12']).rename('NDSI2')
    return img.addBands(ndsi).addBands(ndsi2)

# Function to mask clouds
def maskClouds(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)

# Function to mask edges
def maskEdges(s2_img):
    return s2_img.updateMask(
        s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

#BLOCK2

SOS_yearly_list = []

for year in [2019, 2020, 2021, 2022, 2023]:

    date_ranges = get_date_ranges(year)

    SOS_median = ee.Image.constant(0)

    full_EVI2_collection = ee.ImageCollection([])

    for date_range in date_ranges: #phenology season
        START_DATE = ee.Date(date_range['start_date'])
        END_DATE = ee.Date(date_range['end_date'])
        middle_date = START_DATE.advance((END_DATE.difference(START_DATE, 'days')).divide(2), 'days')

        # Filter images by date range and region
        criteria_a = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE))
        criteria_b = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE),
            ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 40))

        s2Sr_filtered = s2Sr.filter(criteria_b)
        s2Clouds_filtered = s2Clouds.filter(criteria_a)

        # Join S2 SR with cloud probability dataset to add cloud mask
        s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
            primary=s2Sr_filtered,
            secondary=s2Clouds_filtered,
            condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
        )
        s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)
        s2CloudMaskedWithNDSI = s2CloudMasked.map(computeNDSI)
        PPIvis = {'min': -2, 'max': 2, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red'], 'bands': ['PPI']}
        rgbVis = {'min': 0, 'max': 2500, 'bands': ['B4', 'B3', 'B2']}

        images = s2CloudMasked.toList(s2CloudMaskedWithNDSI.size())

        EVI2_images = []

        try:
            for i in range(images.size().getInfo()):

                image = ee.Image(images.get(i))
                date = image.date().format('YYYY-MM-dd').getInfo()
                # Compute PPI for the image
                EVI2_image = computeEVI2(image)
                EVI2_images.append(EVI2_image)

            EVI2_collection = ee.ImageCollection(EVI2_images)

            median_EVI2_image = EVI2_collection.median()
            full_EVI2_collection = full_EVI2_collection.merge(median_EVI2_image)

        except:
          print('exception, block 2', date_range)
          pass

    #BLOCK3

    s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate(f'{year}-01-01', f'{year}-12-31').filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 50))
    s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(region).filterDate(f'{year}-01-01', f'{year}-12-31')
    MAX_CLOUD_PROBABILITY = 65

    s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
            primary=s2Sr,
            secondary=s2Clouds,
            condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
        )
    s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)

    def seasonal_amplitude(collection):
        EVI_collection = collection.map(computeEVI2)
        EVImax = EVI_collection.select('EVI2').reduce(ee.Reducer.max()).rename('EVImax')
        EVImin = EVI_collection.select('EVI2').reduce(ee.Reducer.min()).rename('EVImin')


        return EVImax.subtract(EVImin)

    seas_amp = seasonal_amplitude(s2Sr)
    seas_amp_median = seasonal_amplitude(full_EVI2_collection)

    #BLOCK4

    s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate('2018-01-01', '2023-12-31')
    s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
    MAX_CLOUD_PROBABILITY = 65

    SOS_median = ee.Image.constant(0)

    full_EVI2_collection = ee.ImageCollection([])

    for date_range in date_ranges: #phenology season
        START_DATE = ee.Date(date_range['start_date'])
        END_DATE = ee.Date(date_range['end_date'])
        middle_date = START_DATE.advance((END_DATE.difference(START_DATE, 'days')).divide(2), 'days')

        # Filter images by date range and region
        criteria_a = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE))
        criteria_b = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE),
            ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 40))

        s2Sr_filtered = s2Sr.filter(criteria_b)
        s2Clouds_filtered = s2Clouds.filter(criteria_a)

        # Join S2 SR with cloud probability dataset to add cloud mask
        s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
            primary=s2Sr_filtered,
            secondary=s2Clouds_filtered,
            condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
        )
        s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)
        s2CloudMaskedWithNDSI = s2CloudMasked.map(computeNDSI)
        PPIvis = {'min': -2, 'max': 2, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red'], 'bands': ['PPI']}
        rgbVis = {'min': 0, 'max': 2500, 'bands': ['B4', 'B3', 'B2']}

        images = s2CloudMasked.toList(s2CloudMaskedWithNDSI.size())

        EVI2_images = []

        try:
            for i in range(images.size().getInfo()):

                image = ee.Image(images.get(i))
                date = image.date().format('YYYY-MM-dd').getInfo()
                # Compute PPI for the image
                EVI2_image = computeEVI2(image)
                EVI2_images.append(EVI2_image)

            EVI2_collection = ee.ImageCollection(EVI2_images)

            # Define the threshold for the seasonal amplitude
            threshold = seas_amp_median.multiply(0.25)

            # Compute the median EVI2 image for the entire date range
            median_EVI2_image = EVI2_collection.median()

            # Mask the median EVI2 image based on the thresholded seasonal amplitude
            masked_median_EVI2_image = median_EVI2_image.updateMask(seas_amp_median.gt(threshold))

            masked_median_EVI2_image = masked_median_EVI2_image.set('system:time_start', middle_date.millis())
            day_of_year = ee.Date(masked_median_EVI2_image.getInfo()['properties']['system:time_start']).difference(ee.Date(f'{year}-01-01'), 'days').round()

            # Create a constant image representing the day of year
            constant_day = ee.Image.constant(day_of_year)

            # Update the result image to store the start-of-season day for each pixel
            SOS_median = SOS_median.where(SOS_median.eq(0).And(masked_median_EVI2_image.select('EVI2').gt(threshold)), constant_day)


            full_EVI2_collection = full_EVI2_collection.merge(median_EVI2_image)

        except:
          print('exception, block 4', date_range)
          pass

    SOS_yearly_list.append([year,SOS_median])

## EOS Medians 2019-2023

In [ ]:
import geemap
import folium
import numpy as np

#BLOCK1

s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate('2018-01-01', '2023-12-31')
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
MAX_CLOUD_PROBABILITY = 65

def map_SOS_EOS(image):
    SOS, EOS = find_SOS_EOS(image, threshold=0.2)
    return image.addBands(SOS.rename('SOS')).addBands(EOS.rename('EOS'))

#DVImax_yearly = compute_yearly_DVImax(s2Sr)

# Create a map centered at the region of interest
map = folium.Map(location=[52.03, 41.16], zoom_start=10)

# Function to compute NDSI
def computeNDSI(img):
    ndsi = img.normalizedDifference(['B3', 'B11']).rename('NDSI')
    ndsi2 = img.normalizedDifference(['B3', 'B12']).rename('NDSI2')
    return img.addBands(ndsi).addBands(ndsi2)

# Function to mask clouds
def maskClouds(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)

# Function to mask edges
def maskEdges(s2_img):
    return s2_img.updateMask(
        s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

#BLOCK2

EOS_yearly_list = []

Seasamp_yearly_list = []

for year in [2019, 2020, 2021, 2022, 2023]:

    date_ranges = get_date_ranges(year)

    EOS_median = ee.Image.constant(0)

    full_EVI2_collection = ee.ImageCollection([])

    for date_range in date_ranges: #phenology season
        START_DATE = ee.Date(date_range['start_date'])
        END_DATE = ee.Date(date_range['end_date'])
        middle_date = START_DATE.advance((END_DATE.difference(START_DATE, 'days')).divide(2), 'days')

        # Filter images by date range and region
        criteria_a = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE))
        criteria_b = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE),
            ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 40))

        s2Sr_filtered = s2Sr.filter(criteria_b)
        s2Clouds_filtered = s2Clouds.filter(criteria_a)

        # Join S2 SR with cloud probability dataset to add cloud mask
        s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
            primary=s2Sr_filtered,
            secondary=s2Clouds_filtered,
            condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
        )
        s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)
        s2CloudMaskedWithNDSI = s2CloudMasked.map(computeNDSI)
        PPIvis = {'min': -2, 'max': 2, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red'], 'bands': ['PPI']}
        rgbVis = {'min': 0, 'max': 2500, 'bands': ['B4', 'B3', 'B2']}

        images = s2CloudMasked.toList(s2CloudMaskedWithNDSI.size())

        EVI2_images = []

        try:
            for i in range(images.size().getInfo()):

                image = ee.Image(images.get(i))
                date = image.date().format('YYYY-MM-dd').getInfo()
                # Compute PPI for the image
                EVI2_image = computeEVI2(image)
                EVI2_images.append(EVI2_image)

            EVI2_collection = ee.ImageCollection(EVI2_images)

            median_EVI2_image = EVI2_collection.median()
            full_EVI2_collection = full_EVI2_collection.merge(median_EVI2_image)

        except:
          print('exception, block 2', date_range)
          pass

    #BLOCK3

    s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate(f'{year}-01-01', f'{year}-12-31').filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 50))
    s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(region).filterDate(f'{year}-01-01', f'{year}-12-31')
    MAX_CLOUD_PROBABILITY = 65

    s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
            primary=s2Sr,
            secondary=s2Clouds,
            condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
        )
    s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)

    def seasonal_amplitude(collection):
        EVI_collection = collection.map(computeEVI2)
        EVImax = EVI_collection.select('EVI2').reduce(ee.Reducer.max()).rename('EVImax')
        EVImin = EVI_collection.select('EVI2').reduce(ee.Reducer.min()).rename('EVImin')


        return EVImax.subtract(EVImin)

    seas_amp = seasonal_amplitude(s2Sr)
    seas_amp_median = seasonal_amplitude(full_EVI2_collection)

    #BLOCK4

    s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate('2018-01-01', '2023-12-31')
    s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
    MAX_CLOUD_PROBABILITY = 65

    EOS_median = ee.Image.constant(0)

    full_EVI2_collection = ee.ImageCollection([])

    for date_range in date_ranges[13:]: #phenology season
        START_DATE = ee.Date(date_range['start_date'])
        END_DATE = ee.Date(date_range['end_date'])
        middle_date = START_DATE.advance((END_DATE.difference(START_DATE, 'days')).divide(2), 'days')

        # Filter images by date range and region
        criteria_a = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE))
        criteria_b = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE),
            ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 40))

        s2Sr_filtered = s2Sr.filter(criteria_b)
        s2Clouds_filtered = s2Clouds.filter(criteria_a)

        # Join S2 SR with cloud probability dataset to add cloud mask
        s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
            primary=s2Sr_filtered,
            secondary=s2Clouds_filtered,
            condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
        )
        s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)
        s2CloudMaskedWithNDSI = s2CloudMasked.map(computeNDSI)
        PPIvis = {'min': -2, 'max': 2, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red'], 'bands': ['PPI']}
        rgbVis = {'min': 0, 'max': 2500, 'bands': ['B4', 'B3', 'B2']}

        images = s2CloudMasked.toList(s2CloudMaskedWithNDSI.size())

        EVI2_images = []

        try:
            for i in range(images.size().getInfo()):

                image = ee.Image(images.get(i))
                date = image.date().format('YYYY-MM-dd').getInfo()
                # Compute PPI for the image
                EVI2_image = computeEVI2(image)
                EVI2_images.append(EVI2_image)

            EVI2_collection = ee.ImageCollection(EVI2_images)

            # Define the threshold for the seasonal amplitude
            threshold = seas_amp_median.multiply(0.15)

            # Compute the median EVI2 image for the entire date range
            median_EVI2_image = EVI2_collection.median()

            # Mask the median EVI2 image based on the thresholded seasonal amplitude
            masked_median_EVI2_image = median_EVI2_image.updateMask(seas_amp_median.gt(threshold))

            masked_median_EVI2_image = masked_median_EVI2_image.set('system:time_start', middle_date.millis())
            day_of_year = ee.Date(masked_median_EVI2_image.getInfo()['properties']['system:time_start']).difference(ee.Date(f'{year}-01-01'), 'days').round()

            # Create a constant image representing the day of year
            constant_day = ee.Image.constant(day_of_year)

            # Update the result image to store the start-of-season day for each pixel
            EOS_median = EOS_median.where((masked_median_EVI2_image.select('EVI2').gt(threshold)), constant_day)

            negative_condition = EOS_median.lte(0)

            # Get the maximum EOS value
            max_EOS_value = EOS_median.reduceRegion(
                reducer=ee.Reducer.max(),
                geometry=region,
                scale=10
            ).get('constant')

            max_EOS_constant = ee.Image.constant(max_EOS_value)

            # Set EOS_median to the maximum EOS value where it is negative
            EM = EOS_median.where(negative_condition, max_EOS_constant)

        except:
          print('exception, block 4', date_range)
          pass
    Seasamp_yearly_list.append([year, seas_amp_median])
    EOS_yearly_list.append([year,EM])

## Seasonal Productivity 2019-2023

### Option 2

In [ ]:
import geemap
import folium
import numpy as np
import traceback
import sys

# BLOCK 1

s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate('2018-01-01', '2023-12-31')
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
MAX_CLOUD_PROBABILITY = 65

def map_SOS_EOS(image):
    SOS, EOS = find_SOS_EOS(image, threshold=0.2)
    return image.addBands(SOS.rename('SOS')).addBands(EOS.rename('EOS'))

# DVImax_yearly = compute_yearly_DVImax(s2Sr)

# Create a map centered at the region of interest
map = folium.Map(location=[52.03, 41.16], zoom_start=10)

# Function to compute NDSI
def computeNDSI(img):
    ndsi = img.normalizedDifference(['B3', 'B11']).rename('NDSI')
    ndsi2 = img.normalizedDifference(['B3', 'B12']).rename('NDSI2')
    return img.addBands(ndsi).addBands(ndsi2)

# Function to mask clouds
def maskClouds(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)

# Function to mask edges
def maskEdges(s2_img):
    return s2_img.updateMask(
        s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

# BLOCK 2

Season_len_yearly_list = []
TP_yearly_list = []
threshold1 = 0.2
threshold2 = 0.2

for year in [2019, 2020, 2021, 2022, 2023]:

    date_ranges = get_date_ranges(year)

    SOS_median = ee.Image.constant(0)
    EOS_median = ee.Image.constant(0)

    full_EVI2_collection = ee.ImageCollection([])

    for date_range in date_ranges:  # phenology season
        START_DATE = ee.Date(date_range['start_date'])
        END_DATE = ee.Date(date_range['end_date'])
        middle_date = START_DATE.advance((END_DATE.difference(START_DATE, 'days')).divide(2), 'days')

        # Filter images by date range and region
        criteria_a = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE))
        criteria_b = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE),
            ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 40))

        s2Sr_filtered = s2Sr.filter(criteria_b)
        s2Clouds_filtered = s2Clouds.filter(criteria_a)

        # Join S2 SR with cloud probability dataset to add cloud mask
        s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
            primary=s2Sr_filtered,
            secondary=s2Clouds_filtered,
            condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
        )
        s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)
        s2CloudMaskedWithNDSI = s2CloudMasked.map(computeNDSI)
        PPIvis = {'min': -2, 'max': 2, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red'], 'bands': ['PPI']}
        rgbVis = {'min': 0, 'max': 2500, 'bands': ['B4', 'B3', 'B2']}

        images = s2CloudMasked.toList(s2CloudMaskedWithNDSI.size())

        EVI2_images = []

        try:
            for i in range(images.size().getInfo()):

                image = ee.Image(images.get(i))
                date = image.date().format('YYYY-MM-dd').getInfo()
                # Compute PPI for the image
                EVI2_image = computeEVI2(image)
                EVI2_images.append(EVI2_image)

            EVI2_collection = ee.ImageCollection(EVI2_images)

            median_EVI2_image = EVI2_collection.median()
            full_EVI2_collection = full_EVI2_collection.merge(median_EVI2_image)

        except Exception as e:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            traceback.print_exception(exc_type, exc_value, exc_traceback)
            print('Exception in block 2, date range:', date_range)
            pass

    # BLOCK 3

    s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate(f'{year}-01-01', f'{year}-12-31').filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 50))
    s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(region).filterDate(f'{year}-01-01', f'{year}-12-31')
    MAX_CLOUD_PROBABILITY = 65

    s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
        primary=s2Sr,
        secondary=s2Clouds,
        condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
    )
    s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)

    def seasonal_amplitude(collection):
        EVI_collection = collection.map(computeEVI2)
        EVImax = EVI_collection.select('EVI2').reduce(ee.Reducer.max()).rename('EVImax')
        EVImin = EVI_collection.select('EVI2').reduce(ee.Reducer.min()).rename('EVImin')

        return EVImax.subtract(EVImin)

    seas_amp = seasonal_amplitude(s2Sr)
    seas_amp_median = seasonal_amplitude(full_EVI2_collection)

    # BLOCK 4

    s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate('2018-01-01', '2023-12-31')
    s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
    MAX_CLOUD_PROBABILITY = 65

    SOS_median = ee.Image.constant(0)
    EOS_median = ee.Image.constant(0)

    full_EVI2_collection = ee.ImageCollection([])

    for date_range in date_ranges[13:]:  # phenology season
        START_DATE = ee.Date(date_range['start_date'])
        END_DATE = ee.Date(date_range['end_date'])
        middle_date = START_DATE.advance((END_DATE.difference(START_DATE, 'days')).divide(2), 'days')

        # Filter images by date range and region
        criteria_a = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE))
        criteria_b = ee.Filter.And(
            ee.Filter.bounds(region),
            ee.Filter.date(START_DATE, END_DATE),
            ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 40))

        s2Sr_filtered = s2Sr.filter(criteria_b)
        s2Clouds_filtered = s2Clouds.filter(criteria_a)

        # Join S2 SR with cloud probability dataset to add cloud mask
        s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
            primary=s2Sr_filtered,
            secondary=s2Clouds_filtered,
            condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
        )
        s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)
        s2CloudMaskedWithNDSI = s2CloudMasked.map(computeNDSI)
        PPIvis = {'min': -2, 'max': 2, 'palette': ['cyan', 'green', 'yellow', 'orange', 'red'], 'bands': ['PPI']}
        rgbVis = {'min': 0, 'max': 2500, 'bands': ['B4', 'B3', 'B2']}

        images = s2CloudMasked.toList(s2CloudMaskedWithNDSI.size())

        EVI2_images = []

        try:
            for i in range(images.size().getInfo()):

                image = ee.Image(images.get(i))
                date = image.date().format('YYYY-MM-dd').getInfo()
                # Compute PPI for the image
                EVI2_image = computeEVI2(image)
                EVI2_images.append(EVI2_image)

            EVI2_collection = ee.ImageCollection(EVI2_images)

            # Define the threshold for the seasonal amplitude
            threshold_eos = seas_amp_median.multiply(0.15)

            # Compute the median EVI2 image for the entire date range
            median_EVI2_image = EVI2_collection.median()
            print('1', EVI2_collection.size().getInfo())

            # Mask the median EVI2 image based on the thresholded seasonal amplitude
            masked_median_EVI2_image = median_EVI2_image.updateMask(seas_amp_median.gt(threshold_eos))

            masked_median_EVI2_image = masked_median_EVI2_image.set('system:time_start', middle_date.millis())
            day_of_year = ee.Date(masked_median_EVI2_image.getInfo()['properties']['system:time_start']).difference(ee.Date(f'{year}-01-01'), 'days').round()

            # Create a constant image representing the day of year
            constant_day = ee.Image.constant(day_of_year)

            # Update the result image to store the start-of-season day for each pixel
            EOS_median = EOS_median.where((masked_median_EVI2_image.select('EVI2').gt(threshold_eos)), constant_day)

            negative_condition = EOS_median.lte(0)

            # Get the maximum EOS value
            max_EOS_value = EOS_median.reduceRegion(
                reducer=ee.Reducer.max(),
                geometry=region,
                scale=10
            ).get('constant')

            max_EOS_constant = ee.Image.constant(max_EOS_value)

            # Set EOS_median to the maximum EOS value where it is negative
            EM = EOS_median.where(negative_condition, max_EOS_constant)

            threshold_sos = seas_amp_median.multiply(0.25)

            # Compute the median EVI2 image for the entire date range
            median_EVI2_image = EVI2_collection.median()
            #print('2', EVI2_collection.size().getInfo())

            # Mask the median EVI2 image based on the thresholded seasonal amplitude
            masked_median_EVI2_image = median_EVI2_image.updateMask(seas_amp_median.gt(threshold_sos))

            masked_median_EVI2_image_SOS = masked_median_EVI2_image.set('system:time_start', middle_date.millis())
            #print(type(masked_median_EVI2_image_SOS))
            day_of_year = ee.Date(masked_median_EVI2_image_SOS.getInfo()['properties']['system:time_start']).difference(ee.Date(f'{year}-01-01'), 'days').round()

            # Create a constant image representing the day of year
            constant_day = ee.Image.constant(day_of_year)

            # Update the result image to store the start-of-season day for each pixel
            SOS_median = SOS_median.where(SOS_median.eq(0).And(masked_median_EVI2_image_SOS.select('EVI2').gt(threshold_sos)), constant_day)

            Season_len = EM.subtract(SOS_median)

        except Exception as e:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            traceback.print_exception(exc_type, exc_value, exc_traceback)
            #print('Exception in block 4, date range:', date_range)
            pass

    Season_len_yearly_list.append(Season_len)
'''
    # Get the median SOS date as a constant value
    median_sos = SOS_median.reduceRegion(ee.Reducer.median(), region, 10).get('constant')

    # Get the median EOS date as a constant value
    median_eos = EOS_median.reduceRegion(ee.Reducer.median(), region, 10).get('constant')

    # Convert the median SOS and EOS dates into actual date objects
    median_sos_date = ee.Date.fromYMD(year, 1, 1).advance(median_sos, 'day')
    median_eos_date = ee.Date.fromYMD(year, 1, 1).advance(median_eos, 'day')

    filtered_images = EVI2_collection.filterDate(median_sos_date, median_eos_date)

    # Define a function to replace masked values with the mean of neighboring values
    def fillMask(image):
        # Replace masked values with the mean of neighboring values
        filled_image = image.unmask().focal_mean(radius=1, kernelType='circle', units='pixels')
        return filled_image

    # Map the fillMask function over the filtered images
    filtered_images_filled = filtered_images.map(fillMask)

    # Compute the sum of EVI2 values for each image
    sum_images = filtered_images_filled.sum()

    TP_yearly_list.append(sum_images)

# Print the results
for year, season_len, tp in zip([2019, 2020, 2021, 2022, 2023], Season_len_yearly_list, TP_yearly_list):
    print(f"Year: {year}")
    print(f"Season Length: {season_len.getInfo()}")
    print(f"Total Productivity: {tp.getInfo()}")
'''

### Option 2

In [ ]:
def computeEVI2(image):
    """Compute the EVI2 index for a given image."""
    NIR = image.select('B8')  # Assuming NIR band is labeled as B8
    red = image.select('B4')  # Assuming red band is labeled as B4
    DVI = NIR.subtract(red).rename('DVI')
    EVI2 = ee.Image(2.5).multiply(DVI) \
        .divide(NIR.add(ee.Image(2.4).multiply(red)).add(ee.Image(1))).rename('EVI2')
    return image.addBands(EVI2)

# Function to mask clouds
def maskClouds(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)

# Function to mask edges
def maskEdges(s2_img):
    return s2_img.updateMask(
        s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

# Function to compute the mean EVI2 value from April 1 to December 1
def compute_mean_EVI2(year, threshold):
    start_date = f'{year}-04-01'
    end_date = f'{year}-12-01'

    # Load Sentinel-2 image collection for the given period
    s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(region).filterDate(start_date, end_date).filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 80))
    s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(region).filterDate(start_date, end_date)

    # Join S2 SR with cloud probability dataset to add cloud mask
    s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
        primary=s2Sr,
        secondary=s2Clouds,
        condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
    )

    s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskEdges).map(maskClouds)

    EVI2_images = s2CloudMasked.map(lambda img: computeEVI2(img).select('EVI2').updateMask(computeEVI2(img).select('EVI2').gte(threshold)))

    mean_EVI2 = EVI2_images.mean().clip(region).rename('mean_EVI2')

    return mean_EVI2

# List to store results
TP_yearly_list = []

# Years to process
years = [2019, 2020, 2021, 2022, 2023]
threshold = 0.2

# Process each year
for k, year in enumerate(years):
    mean_EVI2 = compute_mean_EVI2(year, threshold)
    season_length = season_length_list[k][1]
    total_productivity = mean_EVI2.multiply(season_length).rename('total_productivity')
    TP_yearly_list.append([total_productivity, year])
    print(f"Computed total productivity for year {year}")

## Season Len

In [ ]:
import ee

# Initialize the Earth Engine module
ee.Initialize()

# Function to calculate the season length
def calculate_season_length(sos_eos_tuple):
    year, sos_image = sos_eos_tuple[0]
    eos_image = sos_eos_tuple[1][1]  # Assuming corresponding images match by index
    season_length = eos_image.subtract(sos_image)
    # Replace negative values with 0 using the max function
    season_length = season_length.max(0)
    return (season_length, year)

# Pair up corresponding SOS and EOS images by year
paired_images = list(zip(SOS_yearly_list, EOS_yearly_list))

# Calculate the season length for each pair
season_length_list = [calculate_season_length(pair) for pair in paired_images]

# Print or visualize the results
for season_length, year in season_length_list:
    print(f"Year: {year}")
    season_length_info = season_length.getInfo()
    min_value = season_length_info['bands'][0]['data_type']['min']
    max_value = season_length_info['bands'][0]['data_type']['max']
    print(f"Season Length Image Min Value: {min_value}, Max Value: {max_value}")

## Export images to Drive

In [ ]:
# Export SOS images
for year, image in SOS_yearly_list:
    print(type(image))
    export_params = {
      'image': image,
      'description': f'{year}, SOS',  # Specify the file name
      'folder': 'GEE_images',  # Specify the folder in your Google Drive
      'scale': 10,  # Adjust the scale as needed
      'region': region,  # Define the region of interest
      'maxPixels': 1e13  # Specify the maximum number of pixels
    }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
    print(f'exporing SOS for year {year}')
    time.sleep(100)


# Export EOS images
for year, image in EOS_yearly_list:
    export_params = {
      'image': image,
      'description': f'{year}, EOS',  # Specify the file name
      'folder': 'GEE_images',  # Specify the folder in your Google Drive
      'scale': 10,  # Adjust the scale as needed
      'region': region,  # Define the region of interest
      'maxPixels': 1e13  # Specify the maximum number of pixels
    }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
    print(f'exporing EOS for year {year}')
    time.sleep(100)

# Export Seasamp images
for year, image in Seasamp_yearly_list:
    export_params = {
      'image': image,
      'description': f'{year}, Seasamp',  # Specify the file name
      'folder': 'GEE_images',  # Specify the folder in your Google Drive
      'scale': 10,  # Adjust the scale as needed
      'region': region,  # Define the region of interest
      'maxPixels': 1e13  # Specify the maximum number of pixels
    }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
    print(f'exporing Seasamp for year {year}')
    time.sleep(100)

# Export Greenup images
for year, image in greenup_yearly_list:
    export_params = {
      'image': image,
      'description': f'{year}, Greenup',  # Specify the file name
      'folder': 'GEE_images',  # Specify the folder in your Google Drive
      'scale': 10,  # Adjust the scale as needed
      'region': region,  # Define the region of interest
      'maxPixels': 1e13  # Specify the maximum number of pixels
    }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
    print(f'exporing greenup for year {year}')
    time.sleep(200)

# Export Greendown images
for year, image in greendown_yearly_list:
    export_params = {
      'image': image,
      'description': f'{year}, Greendown',  # Specify the file name
      'folder': 'GEE_images',  # Specify the folder in your Google Drive
      'scale': 10,  # Adjust the scale as needed
      'region': region,  # Define the region of interest
      'maxPixels': 1e13  # Specify the maximum number of pixels
    }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
    print(f'exporing Greendown for year {year}')
    time.sleep(100)

# Export TP images
for image, year in TP_yearly_list:
    export_params = {
      'image': image,
      'description': f'{year}, TP',  # Specify the file name
      'folder': 'GEE_images',  # Specify the folder in your Google Drive
      'scale': 10,  # Adjust the scale as needed
      'region': region,  # Define the region of interest
      'maxPixels': 1e13  # Specify the maximum number of pixels
    }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
    print(f'exporing TP for year {year}')
    time.sleep(100)

# Export Season Length images
for image, year in season_length_list:
    export_params = {
      'image': image,
      'description': f'{year}, season_length',  # Specify the file name
      'folder': 'GEE_images',  # Specify the folder in your Google Drive
      'scale': 10,  # Adjust the scale as needed
      'region': region,  # Define the region of interest
      'maxPixels': 1e13  # Specify the maximum number of pixels
    }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
    print(f'exporing season_length for year {year}')
    time.sleep(100)
